<a href="https://colab.research.google.com/github/JimKing100/nfl-test/blob/master/predictions-final/Offense_Predictions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Installs
%%capture
!pip install pmdarima
!pip install category_encoders==2.0.0

In [2]:
# Imports
import numpy as np
import pandas as pd
from statsmodels.tsa.arima_model import ARIMA
import pmdarima as pm
from sklearn import preprocessing

import category_encoders as ce

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
# Import data
players_df = pd.read_csv('https://raw.githubusercontent.com/JimKing100/nfl-test/master/data-actuals/players_full.csv')
original_df = pd.read_csv('https://raw.githubusercontent.com/JimKing100/nfl-test/master/data-actuals/actuals_offense.csv')
kickers_df = pd.read_csv('https://raw.githubusercontent.com/JimKing100/nfl-test/master/data-revised/rookies_non_kicker.csv')
offense_df = pd.read_csv('https://raw.githubusercontent.com/JimKing100/nfl-test/master/data-revised/rookies_non_offense.csv')
bye_df = pd.read_csv('https://raw.githubusercontent.com/JimKing100/nfl-test/master/data-revised/bye.csv')
injury_df = pd.read_csv('https://raw.githubusercontent.com/JimKing100/nfl-test/master/data-revised/injury.csv')
player_df = pd.concat([kickers_df, offense_df], ignore_index=True)

In [4]:
# The dataframe of actual offensive points for each game from 2000-2019
original_df.head()

,player,first,last,name,position1,2000-game1,2000-game2,2000-game3,2000-game4,2000-game5,2000-game6,2000-game7,2000-game8,2000-game9,2000-game10,2000-game11,2000-game12,2000-game13,2000-game14,2000-game15,2000-game16,2001-game1,2001-game2,2001-game3,2001-game4,2001-game5,2001-game6,2001-game7,2001-game8,2001-game9,2001-game10,2001-game11,2001-game12,2001-game13,2001-game14,2001-game15,2001-game16,2002-game1,2002-game2,2002-game3,...,2017-game9,2017-game10,2017-game11,2017-game12,2017-game13,2017-game14,2017-game15,2017-game16,2018-game1,2018-game2,2018-game3,2018-game4,2018-game5,2018-game6,2018-game7,2018-game8,2018-game9,2018-game10,2018-game11,2018-game12,2018-game13,2018-game14,2018-game15,2018-game16,2019-game1,2019-game2,2019-game3,2019-game4,2019-game5,2019-game6,2019-game7,2019-game8,2019-game9,2019-game10,2019-game11,2019-game12,2019-game13,2019-game14,2019-game15,2019-game16
0,AD-0100,Andy,Dalton,Andy Dalton,QB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,15.6,18.56,19.16,19.26,7.64,0.52,11.42,21.68,16.52,26.60,14.08,24.78,12.92,17.16,7.92,20.20,6.92,19.34,8.10,0.00,0.00,0.00,0.00,0.00,21.72,19.64,16.06,3.64,19.98,13.60,18.34,17.16,0.00,0.00,0.00,13.32,9.58,2.14,33.84,16.60
1,AM-1150,A.J.,McCarron,A.J. McCarron,QB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.00,0.00,0.00,1.88,0.76,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.32,0.00,0.00,-0.20,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,17.90
2,AR-1300,Aaron,Rodgers,Aaron Rodgers,QB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.00,0.00,0.00,0.00,21.90,0.00,0.00,25.94,16.04,20.90,15.02,28.68,28.40,16.94,19.16,18.06,22.58,11.92,15.32,20.24,13.46,43.88,1.04,12.92,14.36,14.30,25.48,9.42,18.32,44.76,28.10,12.94,10.02,9.46,28.12,11.40,14.42,9.34,19.02
3,AT-0160,Alex,Tanney,Alex Tanney,QB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.04,0.00,0.00
4,BA-0375,Brandon,Allen,Brandon Allen,QB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,18.02,13.40,2.08,0.00,0.00,0.00,0.00,0.00


In [5]:
# The dataframe of all 2019 offensive players (kickers and offense)
player_df.head()

,player,first,last,name,position1,start
0,AV-0400,Adam,Vinatieri,Adam Vinatieri,K,1996
1,MB-4600,Matt,Bryant,Matt Bryant,K,2002
2,RG-1500,Robbie,Gould,Robbie Gould,K,2005
3,MN-0800,Mike,Nugent,Mike Nugent,K,2005
4,SG-0800,Stephen,Gostkowski,Stephen Gostkowski,K,2006


In [6]:
# The dataframe of bye weeks for 2019 teams
bye_df.head()

,player,first,last,name,position1,bye-week
0,SF,San Francisco,49ers,San Francisco 49ers,DF,4
1,CHI,Chicago,Bears,Chicago Bears,DF,6
2,CIN,Cincinnati,Bengals,Cincinnati Bengals,DF,9
3,BUF,Buffalo,Bills,Buffalo Bills,DF,6
4,DEN,Denver,Broncos,Denver Broncos,DF,10


In [7]:
# The dataframe of injuries for 2019 players
injury_df.head()

,uid,gid,year,week,player,first,last,name,position1,team,details,pstat,gstat
0,29844,5194,2019,10,AA-0581,Antony,Auclair,Antony Auclair,TE,TB,Toe,NaN,IR
1,30530,5213,2019,11,AA-0581,Antony,Auclair,Antony Auclair,TE,TB,Toe,NaN,IR
2,31097,5228,2019,12,AA-0581,Antony,Auclair,Antony Auclair,TE,TB,Toe,NaN,IR
3,31629,5242,2019,13,AA-0581,Antony,Auclair,Antony Auclair,TE,TB,Toe,NaN,IR
4,32187,5256,2019,14,AA-0581,Antony,Auclair,Antony Auclair,TE,TB,Toe,NaN,IR


In [0]:
# Add a row to the final_df dataframe
# Each row represents the predicted points for each player
def add_row(df, p, f, l, n, pos, wc, wp, wa, cur, pred, act):

    df = df.append({'player': p,
                    'first': f,
                    'last': l,
                    'name': n,
                    'position': pos,
                    wc: cur,
                    wp: pred,
                    wa: act
                    }, ignore_index=True)
    
    return df

In [0]:
# The main code for iterating through the player(offense and kicker) list, calculating the points and adding the rows
# to the final_df dataframe.
def main(w):
    week_cname = 'week' + str(w) + '-cur'
    week_pname = 'week' + str(w) + '-pred'
    week_aname = 'week' + str(w) + '-act'

    column_names = ['player',
                    'first',
                    'last',
                    'name',
                    'position',
                    week_cname, 
                    week_pname,
                    week_aname
                     ]

    player_list = offense_df['player'].tolist()
    end_col = 308 + w

    week_df = pd.DataFrame(columns = column_names)

    for player in player_list:

        first = player_df['first'].loc[(player_df['player']==player)].iloc[0]
        last = player_df['last'].loc[(player_df['player']==player)].iloc[0]
        name = player_df['name'].loc[(player_df['player']==player)].iloc[0]
        position1 = player_df['position1'].loc[(player_df['player']==player)].iloc[0]
        start_year = player_df['start'].loc[(player_df['player']==player)].iloc[0]
        row = original_df.index[(original_df['player']==player)][0]
        team = players_df['cteam'].loc[(players_df['player']==player)].iloc[0]

        injury_weeks = injury_df['week'].loc[(injury_df['player']==player)]
        if (len(injury_weeks)==0):
            injury_week = 18
        else:
            injury_week = min(injury_weeks)

        bye_week = bye_df['bye-week'].loc[(bye_df['player']==team)].iloc[0]

        if start_year < 2000:
            start_year = 2000
        col = ((start_year - 2000) * 16) + 5

        if w > bye_week:
            new_col = end_col - 1
        else:
            new_col = end_col

        train_data = original_df.iloc[row, col:new_col]

        current = original_df.iloc[row, 309:new_col]
        cur_points = current.sum()

        actuals = original_df.iloc[row, new_col:325]
        act_points = actuals.sum()

        print(player, train_data.sum())

        if (start_year < 2016) & (player != 'GG-0310') & (train_data.sum() > 1):
            # ARIMA model
            model = pm.auto_arima(train_data, start_p=1, start_q=1,
                                  test='adf',       # use adftest to find optimal 'd'
                                  max_p=3, max_q=3, # maximum p and q
                                  m=1,              # frequency of series
                                  d=None,           # let model determine 'd'
                                  seasonal=False,   # No Seasonality
                                  start_P=0, 
                                  D=0, 
                                  trace=False,
                                  error_action='ignore',  
                                  suppress_warnings=True, 
                                  stepwise=True)
    
            # Forecast
            if w > bye_week:
                n_periods = 17 - w + 1
            else:
                n_periods = 17 - w
            fc = model.predict(n_periods=n_periods, return_conf_int=False)
            index_of_fc = np.arange(len(train_data), len(train_data)+n_periods)
            fc_series = pd.Series(fc, index=index_of_fc)
            if (injury_week <= w):
                pred_points = 0
            else:
                pred_points = fc_series.sum()
        else:
            pred_points = train_data.mean() * (18 - w)

        week_df = add_row(week_df, player, first, last, name, position1, week_cname,
                          week_pname, week_aname, cur_points, pred_points, act_points)
    return week_df

In [10]:
week = 17
final_df = main(week)

TB-2300 5025.200000000003
DB-3800 5038.42
JM-2900 1084.8400000000001
BR-1100 3588.3
EM-0200 3323.6199999999985
MS-0200 1441.08
PR-0300 3623.099999999998
AR-1300 3665.920000000001
RF-0500 2155.04
MM-4700 443.07999999999987
JF-1900 2365.2799999999997
MR-2500 3217.92
BH-1900 569.4599999999999
CD-0300 90.19999999999999
MS-4100 2585.2999999999997
CM-1500 367.5799999999999
AD-0100 2080.6200000000003
CN-0500 2519.539999999999
TT-0500 850.7800000000001
AT-0160 8.0
CK-0250 848.2800000000002
KC-2350 1584.6399999999996
NF-0250 755.0400000000001
RG-1850 708.42
RT-0150 1520.42
RW-3850 2460.48
MB-0450 98.80000000000001
MG-0850 310.52000000000004
RG-1885 0.62
AM-1150 59.02
BB-2425 1193.5599999999995
DC-0725 1407.7200000000005
GG-0310 1.2999999999999998
JG-0650 423.3800000000001
TB-2650 481.21999999999997
BH-2300 130.38
JW-5650 1232.3400000000001
MM-0912 958.4199999999998
SM-0425 4.62
TS-1450 318.86000000000007
BA-0375 33.5
CW-1425 960.18
DP-2037 1158.5600000000004
JB-5725 458.1
JD-2850 143.2200000000

In [11]:
final_df.head(50)

,player,first,last,name,position,week17-cur,week17-pred,week17-act
0,TB-2300,Tom,Brady,Tom Brady,QB,250.84,15.294514,14.84
1,DB-3800,Drew,Brees,Drew Brees,QB,204.74,0.000000,23.02
2,JM-2900,Josh,McCown,Josh McCown,QB,0.76,1.133766,0.00
3,BR-1100,Ben,Roethlisberger,Ben Roethlisberger,QB,12.74,0.000000,0.00
4,EM-0200,Eli,Manning,Eli Manning,QB,56.38,7.416560,0.00
5,MS-0200,Matt,Schaub,Matt Schaub,QB,32.90,1.517098,0.00
6,PR-0300,Philip,Rivers,Philip Rivers,QB,223.76,15.104800,16.74
7,AR-1300,Aaron,Rodgers,Aaron Rodgers,QB,263.36,14.418765,19.02
8,RF-0500,Ryan,Fitzpatrick,Ryan Fitzpatrick,QB,218.16,20.018173,24.30
9,MM-4700,Matt,Moore,Matt Moore,QB,45.26,2.437799,0.00


In [0]:
week_pred = 'week' + str(week) + '-pred'
week_act = 'week' + str(week) + '-act'
week_diff = 'week' + str(week) + '-diff'
week_pct = 'week' + str(week) + '-pct'

In [0]:
# The final_df dataframe
final_df[week_diff] = final_df[week_pred] - final_df[week_act]
final_df[week_pct] = final_df[week_diff] / final_df[week_pred]
final_df[week_pred] = final_df[week_pred].astype(int)

In [14]:
# Calculate the metrics
pred_median_error = final_df[week_pct].median()

print('Median Error - %.4f%%' % (pred_median_error * 100))

Median Error - 67.7569%


In [15]:
final_df.head(50)

,player,first,last,name,position,week17-cur,week17-pred,week17-act,week17-diff,week17-pct
0,TB-2300,Tom,Brady,Tom Brady,QB,250.84,15,14.84,0.454514,0.029717
1,DB-3800,Drew,Brees,Drew Brees,QB,204.74,0,23.02,-23.020000,-inf
2,JM-2900,Josh,McCown,Josh McCown,QB,0.76,1,0.00,1.133766,1.000000
3,BR-1100,Ben,Roethlisberger,Ben Roethlisberger,QB,12.74,0,0.00,0.000000,NaN
4,EM-0200,Eli,Manning,Eli Manning,QB,56.38,7,0.00,7.416560,1.000000
5,MS-0200,Matt,Schaub,Matt Schaub,QB,32.90,1,0.00,1.517098,1.000000
6,PR-0300,Philip,Rivers,Philip Rivers,QB,223.76,15,16.74,-1.635200,-0.108257
7,AR-1300,Aaron,Rodgers,Aaron Rodgers,QB,263.36,14,19.02,-4.601235,-0.319114
8,RF-0500,Ryan,Fitzpatrick,Ryan Fitzpatrick,QB,218.16,20,24.30,-4.281827,-0.213897
9,MM-4700,Matt,Moore,Matt Moore,QB,45.26,2,0.00,2.437799,1.000000


In [0]:
# Save the results to .csv file
file_name = '/content/week' + str(week) + '-pred-offense-norookies.csv'
final_df.to_csv(file_name, index=False)